In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sb
from sklearn.model_selection import KFold
from sklearn import linear_model as lm

#import our data train.csv
donnees = pd.read_csv('train.csv')
macro = pd.read_csv('macro.csv')
train = pd.merge(donnees, macro, on='timestamp')

#nbre ligne,nbre colonne
print(donnees.shape)

featureFilter = lambda feature: True

features = list(filter(featureFilter, train.columns))
prices = train['price_doc']
correlations = list(map(lambda feature: prices.corr(train[feature], 'spearman'), features))
display = pd.DataFrame({'feature': features, 'correlation': correlations}).sort_values(by='correlation')
plt.figure(figsize=(8, 40))
sb.barplot(data=display, orient='h', x='correlation', y='feature')

def error(actual, predicted):
    return np.sqrt(np.sum(np.square(np.log1p(actual)-np.log1p(predicted)))/len(actual))


#features = ['full_sq','life_sq','floor','max_floor','material,build_year','num_room,kitch_sq','state,product_type','sub_area,area_m','raion_popul','green_zone_part','indust_part','children_preschool','preschool_quota','preschool_education_centers_raion','children_school','school_quota','school_education_centers_raion','school_education_centers_top_20_raion','hospital_beds_raion','healthcare_centers_raion','university_top_20_raion','sport_objects_raion','additional_education_raion','culture_objects_top_25','culture_objects_top_25_raion','shopping_centers_raion'
 #          ]

features = ['num_room', 'full_sq', 'life_sq', 
            'cafe_count_3000','cafe_count_3000_price_1000','cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 
            'office_count_5000', 'sport_count_3000','sport_count_5000',
            'cafe_count_2000', 'cafe_count_2000_price_1000','cafe_count_2000_price_2500','cafe_count_2000_price_1500', 
            'cafe_count_5000','cafe_count_5000_price_2500','cafe_count_5000_price_1500','cafe_count_1500'
           ]
test=pd.read_csv('test.csv')

train2 = train.head(10000)

test = test[features]
test2=test[features].fillna(method = 'bfill').values
X = train2[features].fillna('bfill').values
Y = train2['price_doc'].values


#fold = KFold(n_splits=7)
#errors = []

#for trainIndices, testIndices in fold.split(X):
"""
    trainXi = X[trainIndices]
    testXi = X[testIndices]
    trainYi = Y[trainIndices]
    testYi = Y[testIndices]
"""    
lasso = lm.Lasso()
lasso.fit(X, Y)
Ypred = lasso.predict(test2)
print(Ypred)
    #errors.append(error(testYi, Ypred))
    
#print(np.average(errors))


id_test = range(30474 , 38136)
output = pd.DataFrame({'id': id_test, 'price_doc': Ypred})
output.head()
output.to_csv('submission.csv', index=False)
